In [1]:
import time
from datetime import datetime
import os
import sys
import numpy as np
import matplotlib
%matplotlib ipympl
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import style

style.use('ggplot')

from pathlib import Path

import astropy.units as u
from astropy import stats
from astropy.io import fits
from astropy.table import Table, hstack, vstack

from astropy.modeling.models import Gaussian2D, Polynomial2D, Moffat2D
from astropy.modeling.fitting import LevMarLSQFitter

from mmtwfs.wfs import *
from mmtwfs.zernike import ZernikeVector
from mmtwfs.telescope import MMT

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%cd /Users/tim/MMT/wfsdat/20180209

/Users/tim/MMT/wfsdat/20180209


In [4]:
hdr_keys = ['DATE-OBS', 'RA', 'DEC', 'AZ', 'EL', 'ROT', 'TEMP', 'FOCUS', 'CHAM_DPT', 'CHAM_T', 'OUT_T', 'WIND_W', 'WDIR_W'
            'EXPTIME', 'AIRMASS', 'OSSTEMP', 'TRANSX', 'TRANSY', 'TILTX', 'TILTY', 'PRESSURE']

In [5]:
file = "wfs_ff_cal_img_2018.0209.033646.fits"
hdulist = fits.open(file)
data = hdulist[-1].data
hdr = {}
for h in hdulist:
    hdr.update(h.header)
mean, median, stddev = stats.sigma_clipped_stats(data, sigma=3.0, iters=None)
data = data - median
data.shape

(516, 532)

In [6]:
spots, fig = wfsfind(data)
fig.show()

FigureCanvasNbAgg()

In [7]:
apsize = 12.
apers = photutils.CircularAperture(
        (spots['xcentroid'], spots['ycentroid']),
        r=apsize
)
masks = apers.to_mask(method="subpixel")

In [8]:
spot_lines = []
gauss_resids = []
moffat_resids = []
for m, s in zip(masks, spots):
    subim = m.cutout(data)
    props = photutils.data_properties(subim)
    tline = hstack([s, props.to_table()])
    tline['filename'] = file
    for k in hdr_keys:
        if k in hdr:
            tline[k] = hdr[k]

    y, x = np.mgrid[:subim.shape[0], :subim.shape[1]]
    sigma = (tline['semimajor_axis_sigma'][0].value + tline['semiminor_axis_sigma'][0].value) / 2.
    
    fitter = LevMarLSQFitter()
    gauss_model = Gaussian2D(
        amplitude=subim.max(),
        x_mean=subim.shape[1]/2.,
        y_mean=subim.shape[0]/2.,
        x_stddev = sigma,
        y_stddev = sigma
    ) + Polynomial2D(degree=0)
    moffat_model = Moffat2D(
        amplitude=subim.max(),
        x_0=subim.shape[1]/2.,
        y_0=subim.shape[0]/2.,
        gamma=sigma
    ) + Polynomial2D(degree=0)
    gauss_fit = fitter(gauss_model, x, y, subim)
    moffat_fit = fitter(moffat_model, x, y, subim)
    
    gauss_resid = subim - gauss_fit(x, y)
    moffat_resid = subim - moffat_fit(x, y)
    gauss_resids.append(gauss_resid)
    moffat_resids.append(moffat_resid)
    
    gamma = moffat_fit.gamma_0.value
    alpha = moffat_fit.alpha_0.value
    moffat_fwhm = 2. * gamma * np.sqrt(2.**(1./alpha) - 1.)
    gauss_fwhm = 0.5 * (gauss_fit.x_stddev_0.value + gauss_fit.y_stddev_0.value) * stats.gaussian_sigma_to_fwhm
    moment_fwhm = 0.5 * (tline['semimajor_axis_sigma'] + tline['semiminor_axis_sigma']) * stats.gaussian_sigma_to_fwhm
    
    tline['gauss_x'] = gauss_fit.x_mean_0.value
    tline['gauss_y'] = gauss_fit.y_mean_0.value
    tline['gauss_sigx'] = gauss_fit.x_stddev_0.value
    tline['gauss_sigy'] = gauss_fit.y_stddev_0.value
    tline['gauss_amplitude'] = gauss_fit.amplitude_0.value
    tline['gauss_theta'] = gauss_fit.theta_0.value
    tline['gauss_background'] = gauss_fit.c0_0_1.value
    tline['gauss_rms'] = np.nanstd(gauss_resid)
    tline['moffat_amplitude'] = moffat_fit.amplitude_0.value
    tline['moffat_gamma'] = gamma
    tline['moffat_alpha'] = alpha
    tline['moffat_x'] = moffat_fit.x_0_0.value
    tline['moffat_y'] = moffat_fit.y_0_0.value
    tline['moffat_background'] = moffat_fit.c0_0_1.value
    tline['moffat_rms'] = np.nanstd(moffat_resid)
    tline['moment_fwhm'] = moment_fwhm
    tline['gauss_fwhm'] = gauss_fwhm
    tline['moffat_fwhm'] = moffat_fwhm
    spot_lines.append(tline)

In [9]:
t = Table(vstack(spot_lines)).to_pandas()

In [10]:
t.columns

Index(['id_1', 'xcentroid_1', 'ycentroid_1', 'sharpness', 'roundness1',
       'roundness2', 'npix', 'sky', 'peak', 'flux', 'mag', 'id_2',
       'xcentroid_2', 'ycentroid_2', 'sky_centroid', 'sky_centroid_icrs',
       'source_sum', 'source_sum_err', 'background_sum', 'background_mean',
       'background_at_centroid', 'xmin', 'xmax', 'ymin', 'ymax', 'min_value',
       'max_value', 'minval_xpos', 'minval_ypos', 'maxval_xpos', 'maxval_ypos',
       'area', 'equivalent_radius', 'perimeter', 'semimajor_axis_sigma',
       'semiminor_axis_sigma', 'eccentricity', 'orientation', 'ellipticity',
       'elongation', 'covar_sigx2', 'covar_sigxy', 'covar_sigy2', 'cxx', 'cxy',
       'cyy', 'filename', 'DATE-OBS', 'RA', 'DEC', 'AZ', 'EL', 'ROT', 'TEMP',
       'FOCUS', 'CHAM_DPT', 'CHAM_T', 'OUT_T', 'WIND_W', 'AIRMASS', 'OSSTEMP',
       'TRANSX', 'TRANSY', 'TILTX', 'TILTY', 'PRESSURE', 'gauss_x', 'gauss_y',
       'gauss_sigx', 'gauss_sigy', 'gauss_amplitude', 'gauss_theta',
       'gauss_back

In [11]:
t.hist(column='gauss_rms', bins=50, range=(0,100), alpha=0.6)
t.hist(column='moffat_rms', bins=50, range=(0,100), alpha=0.6)
plt.show()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

In [12]:
t

,id_1,xcentroid_1,ycentroid_1,sharpness,roundness1,roundness2,npix,sky,peak,flux,...,moffat_amplitude,moffat_gamma,moffat_alpha,moffat_x,moffat_y,moffat_background,moffat_rms,moment_fwhm,gauss_fwhm,moffat_fwhm
0,1,343.891231,177.179033,0.685763,-0.277652,0.211785,81.0,0.0,1460.0,9.860700,...,1475.549309,2.225404,1.572377,11.857755,12.218349,49.694286,37.936323,12.995311,4.256493,3.312767
1,2,322.699505,178.978707,0.695705,-0.106284,0.047272,81.0,0.0,2414.0,16.652044,...,2454.743760,2.594385,2.065654,11.704007,11.993586,56.659682,32.271733,11.974728,3.840992,3.276416
2,3,239.114439,179.798555,0.623931,0.172228,0.222106,81.0,0.0,1286.0,9.175929,...,1284.534817,2.841356,1.991286,12.120005,11.798046,44.738129,32.034557,12.684916,4.299670,3.666825
3,4,260.071808,179.965432,0.694887,-0.047445,0.156526,81.0,0.0,2282.0,15.710716,...,2274.423357,2.514491,1.914111,12.070868,11.957528,49.926871,35.488470,11.871976,4.034797,3.322092
4,5,281.059003,180.182437,0.714235,-0.002316,0.262550,81.0,0.0,2503.0,17.075334,...,2645.794980,2.237179,1.815707,12.004768,12.208001,56.321602,58.779779,12.000426,3.668695,3.050603
5,6,301.551162,179.617321,0.684324,-0.069609,0.164979,81.0,0.0,1993.0,13.668557,...,2188.836509,-2.189478,1.675180,11.565767,11.654460,54.223940,41.227511,12.381031,3.855663,-3.134905
6,8,322.557576,198.478414,0.630051,-0.245103,0.050019,81.0,0.0,2300.0,16.958352,...,2534.585189,2.910959,2.423137,11.535226,12.497021,63.910475,34.423046,12.161991,3.841482,3.350330
7,9,364.728888,198.022180,0.750641,-0.426837,-0.036181,81.0,0.0,2414.0,15.768149,...,2454.421177,2.153886,1.701887,11.686147,12.068535,56.522948,42.669294,12.310887,3.772869,3.054351
8,11,301.131457,199.402734,0.689294,-0.113977,0.109159,81.0,0.0,2206.0,15.164534,...,2297.998351,2.427395,1.892636,12.168440,12.390787,60.344498,39.367328,12.411019,3.857223,3.228680
9,12,343.410259,198.693893,0.570704,-0.322382,0.436796,81.0,0.0,1677.0,12.009681,...,1678.760978,3.452794,2.186324,12.460509,11.614244,56.053695,56.298739,12.264909,4.946612,4.217807


In [21]:
%matplotlib ipympl
plt.imshow(gauss_resids[10])
plt.show()

FigureCanvasNbAgg()

In [22]:
#%matplotlib ipympl
plt.imshow(moffat_resids[10])
plt.show()